In [ ]:
# TODO: Enter the foldername in your Drive where you have saved the unzipped
# assignment folder, e.g. 'cs231n/assignments/assignment1/'
FOLDERNAME = 'home/ubuntu/Vision-Classifiers/Microsoft-Vision-Classifier/'
assert FOLDERNAME is not None, "[!] Enter the foldername."

# Now that we've mounted your Drive, this ensures that
# the Python interpreter of the Colab VM can load
# python files from within it.
import sys
sys.path.append('/home/ubuntu/Vision-Classifiers/Microsoft-Vision-Classifier')

# This downloads the flowers dataset to your Drive
# if it doesn't already exist.
%cd /$FOLDERNAME/flowers/
!bash get_datasets.sh
%cd /$FOLDERNAME

In [1]:

import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
 
%matplotlib inline  
style.use('fivethirtyeight')
sns.set(style='whitegrid',color_codes=True)


from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score,precision_score,recall_score,confusion_matrix,roc_curve,roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder



import random as rn

import cv2                  
import numpy as np  
from tqdm import tqdm
import os                   
from random import shuffle  
from zipfile import ZipFile
from PIL import Image
import tensorflow
from tensorflow.keras import backend as K
import tensorflow as tf

from tensorflow.keras.utils import to_categorical

ModuleNotFoundError: No module named 'tensorflow.python.tools'

In [ ]:
import os

path = '/home/ubuntu/Vision-Classifiers/Microsoft-Vision-Classifier'
os.listdir('/home/ubuntu/Vision-Classifiers/Microsoft-Vision-Classifier/flowers')



In [ ]:

import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0)
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

def calculate_error(x, y):
  """ returns relative error """
  return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

In [ ]:
import torch
import torchvision
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import numpy as np
from sklearn.linear_model import LogisticRegression
import progressbar
from progressbar import progressbar

from sklearn.neighbors import KNeighborsClassifier

In [ ]:
import microsoftvision

In [ ]:
X=[]
Z=[]
IMG_SIZE=224
FLOWER_DAISY_DIR=f'{path}/flowers/daisy'
FLOWER_SUNFLOWER_DIR=f'{path}/flowers/sunflower'
FLOWER_TULIP_DIR=f'{path}/flowers/tulip'
FLOWER_DANDI_DIR=f'{path}/flowers/dandelion'
FLOWER_ROSE_DIR=f'{path}/flowers/rose'
weights_path= f'{path}/trans-learn-weights/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'


In [ ]:
def assign_label(img,flower_type):
    return flower_type
    

In [ ]:
# def get_image():
#     img = cv2.imread('example.jpg', cv2.IMREAD_COLOR)
#     preprocess = transforms.Compose([
#         transforms.ToPILImage(),
#         transforms.Resize(224),
#         transforms.CenterCrop(224),
#         transforms.ToTensor(),
#         transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
#     ])
#     return preprocess(img).unsqueeze(0) # Unsqueeze only required when there's 1 image in images batch



In [ ]:
def make_train_data(flower_type,DIR):
    for img in tqdm(os.listdir(DIR)):
        label=assign_label(img,flower_type)
        path = os.path.join(DIR,img)
        img = cv2.imread(path,cv2.IMREAD_COLOR)
#         preprocess = transforms.Compose([
#         transforms.ToPILImage(),
#         transforms.Resize(250),
#         transforms.CenterCrop(250),
#         transforms.ToTensor(),
#         transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),])
        #img = preprocess(img).unsqueeze(0) # Unsqueeze only required when there's 1 image in images batch
#         output = cv2.resize(img, (IMG_SIZE,IMG_SIZE))

        

        preprocess = transforms.Compose([  transforms.ToPILImage(),
                                           transforms.Resize(224),
                                           transforms.CenterCrop(224),
                                           transforms.ToTensor(),
                                           transforms.Normalize(mean=[0.406, 0.456, 0.485], std=[0.225, 0.224, 0.229])])

        output=preprocess(img)[[0,1,2],:,:]
        
        
        X.append(np.array(output))
        Z.append(str(label))
        print(output.shape)
        
        
        

In [ ]:
make_train_data('Daisy',FLOWER_DAISY_DIR)
print(len(X))

In [ ]:
make_train_data('Sunflower',FLOWER_SUNFLOWER_DIR)
print(len(X))

In [ ]:
make_train_data('Tulip',FLOWER_TULIP_DIR)
print(len(X))

In [ ]:
make_train_data('Dandelion',FLOWER_DANDI_DIR)
print(len(X))

In [ ]:
make_train_data('Rose',FLOWER_ROSE_DIR)
print(len(X))

In [ ]:
fig,ax=plt.subplots(5,2)
fig.set_size_inches(25,25)
for i in range(5):
    for j in range (2):
        l=rn.randint(0,len(Z))
        ax[i,j].imshow(X[l])
        ax[i,j].set_title('Flower: '+Z[l])
        
plt.tight_layout()
        

In [ ]:

le=LabelEncoder()
Y=le.fit_transform(Z)
Y=to_categorical(Y,5)
X=np.array(X)
X=X/255

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.25,random_state=42)


In [ ]:
model = microsoftvision.models.resnet50(pretrained=True)

model = microsoftvision.models.resnet50(pretrained=True)
features = model(get_image())
print(features.shape)

In [ ]:
model.eval()
model.cuda()

In [ ]:
def get_features(dataset, model):
    all_features = []
    all_labels = []
    import time
    import logging
    import progressbar
    torch.cuda.empty_cache()

    with torch.no_grad():
        for images, labels in progressbar.progressbar(DataLoader(dataset, batch_size=128, num_workers=8)):
            images = images.cuda()
            labels = labels.cuda()
            features = model(images)

            all_features.append(features)
            all_labels.append(labels)

    return torch.cat(all_features).cpu().numpy(), torch.cat(all_labels).cpu().numpy()

In [ ]:
train_features, train_labels = get_features(train_dataset, model)
test_features, test_labels = get_features(test_dataset, model)

In [ ]:

classifier = KNeighborsClassifier(n_neighbors=5, n_jobs=16)

In [ ]:
classifier.fit(train_features, train_labels)
predictions = classifier.predict(test_features)
accuracy = np.mean((test_labels == predictions).astype(np.float)) * 100.
print(f"Accuracy: {accuracy}")

In [ ]:
base_model=VGG16(include_top=False, weights=None,input_shape=(150,150,3), pooling='avg')
 

In [ ]:
base_model.load_weights(weights_path)

In [ ]:
base_model.summary()

In [ ]:
model=Sequential()



model.add(base_model)

model.add(Dense(256,activation='relu'))
model.add(Dense(5,activation='softmax'))



In [ ]:
datagen = ImageDataGenerator(
        featurewise_center=False,  
        samplewise_center=False,  
        featurewise_std_normalization=False,  
        samplewise_std_normalization=False,  
        zca_whitening=False,  
        rotation_range=10,  
        zoom_range = 0.1, 
        width_shift_range=0.2,  
        height_shift_range=0.2,  
        horizontal_flip=True, 
        vertical_flip=False)  


datagen.fit(x_train)

In [ ]:
epochs=50
batch_size=128
red_lr=ReduceLROnPlateau(monitor='val_acc', factor=0.1, epsilon=0.0001, patience=2, verbose=1)

In [ ]:
model.summary()

In [ ]:
base_model.trainable=False 

In [ ]:
model.compile(optimizer=Adam(lr=1e-4),loss='categorical_crossentropy',metrics=['accuracy'])



In [ ]:
gpus= tf.config.experimental.list_physical_devices('GPU')

History = model.fit_generator(datagen.flow(x_train,y_train, batch_size=batch_size),
                              epochs = 50, validation_data = (x_test,y_test),
                              verbose = 1, steps_per_epoch=x_train.shape[0] // batch_size)

In [ ]:
for i in range (len(base_model.layers)):
    print (i,base_model.layers[i])
  
for layer in base_model.layers[15:]:
    layer.trainable=True
for layer in base_model.layers[0:15]:
    layer.trainable=False
  

In [ ]:
model.compile(optimizer=Adam(lr=1e-4),loss='categorical_crossentropy',metrics=['accuracy'])



In [ ]:
History = model.fit_generator(datagen.flow(x_train,y_train, batch_size=batch_size),
                              epochs = 50, validation_data = (x_test,y_test),
                              verbose = 1, steps_per_epoch=x_train.shape[0] // batch_size)

In [ ]:
for i in range (len(base_model.layers)):
    print (i,base_model.layers[i])
  
for layer in base_model.layers[11:]:
    layer.trainable=True
for layer in base_model.layers[0:11]:
    layer.trainable=False
  

In [ ]:
model.compile(optimizer=Adam(lr=1e-4),loss='categorical_crossentropy',metrics=['accuracy'])



In [ ]:
History = model.fit_generator(datagen.flow(x_train,y_train, batch_size=batch_size),
                              epochs = 50, validation_data = (x_test,y_test),
                              verbose = 1, steps_per_epoch=x_train.shape[0] // batch_size)

In [ ]:
plt.plot(History.history['loss'])
plt.plot(History.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['train', 'test'])
plt.show()

In [ ]:
plt.plot(History.history['accuracy'])
plt.plot(History.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train', 'test'])
plt.show()

In [ ]:
model=Sequential()
model.add(base_model)

model.add(Dense(256,activation='relu'))
model.add(BatchNormalization())
model.add(Dense(5,activation='softmax'))


for layer in base_model.layers:
    layer.trainable=True

model.compile(optimizer=Adam(lr=1e-4),loss='categorical_crossentropy',metrics=['accuracy'])



In [ ]:
History = model.fit_generator(datagen.flow(x_train,y_train, batch_size=batch_size),
                              epochs = 50, validation_data = (x_test,y_test),
                              verbose = 1, steps_per_epoch=x_train.shape[0] // batch_size)

In [ ]:
plt.plot(History.history['accuracy'])
plt.plot(History.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train', 'test'])
plt.show()

In [ ]:

pred=model.predict(x_test)
pred_digits=np.argmax(pred,axis=1)

In [ ]:

i=0
prop_class=[]
mis_class=[]

for i in range(len(y_test)):
    if(np.argmax(y_test[i])==pred_digits[i]):
        prop_class.append(i)
    if(len(prop_class)==8):
        break

i=0
for i in range(len(y_test)):
    if(not np.argmax(y_test[i])==pred_digits[i]):
        mis_class.append(i)
    if(len(mis_class)==8):
        break

In [ ]:
warnings.filterwarnings('ignore')

count=0
fig,ax=plt.subplots(4,2)
fig.set_size_inches(15,15)
for i in range (4):
    for j in range (2):
        ax[i,j].imshow(x_test[prop_class[count]])
        ax[i,j].set_title("Predicted Flower : "+str(le.inverse_transform([pred_digits[prop_class[count]]]))+"\n"+"Actual Flower : "+str(le.inverse_transform(np.argmax([y_test[prop_class[count]]]))))
        plt.tight_layout()
        count+=1

In [ ]:
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

count=0
fig,ax=plt.subplots(4,2)
fig.set_size_inches(15,15)
for i in range (4):
    for j in range (2):
        ax[i,j].imshow(x_test[mis_class[count]])
        ax[i,j].set_title("Predicted Flower : "+str(le.inverse_transform([pred_digits[mis_class[count]]]))+"\n"+"Actual Flower : "+str(le.inverse_transform(np.argmax([y_test[mis_class[count]]]))))
        plt.tight_layout()
        count+=1